In [13]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
data = {
    "ID": [1, 2, 3],
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 22]
}
your_dataframe = pd.DataFrame(data)
create_statement = return_create_statement_from_dataframe(your_dataframe, "test_schema", "table_name")


In [15]:
from db_handler import create_connection, execute_query
db_session = create_connection()
print(db_session.closed)

0


In [16]:
execute_query(conn= db_session, query= create_statement)